# Data Cleaning

In [2]:
import pandas as pd
from src.core.paths import ALTERED_DATA_TYPES, TEMPORARY_DATA, CLEANED_DATA
from src.core.miscellaneous import find_first_nan, use_primary_geocoder, use_secondary_geocoder, add_coordinates_to_dataframe, find_rows_with_zeros, reveal_final_unknown_lats, concat_to_parquet

## 2014 - 2017

In [ ]:
trips_2014 = pd.read_parquet(ALTERED_DATA_TYPES/"2014 - 2017/0.parquet")
trips_2015 = pd.read_parquet(ALTERED_DATA_TYPES/"2014 - 2017/1.parquet")
trips_2016 = pd.read_parquet(ALTERED_DATA_TYPES/"2014 - 2017/2.parquet")
trips_2017 = pd.read_parquet(ALTERED_DATA_TYPES/"2014 - 2017/3.parquet")

pre_2018 = [trips_2014, trips_2015, trips_2016, trips_2017]

##### Dropping Irrelevant Columns

2014 : 'starttime', 'stoptime'
2015 : 'starttime', 'stoptime'
2016 : 'starttime', 'stoptime'
2017 : 'start_time', 'end_time'

###### All four of these dataframes have the same column names. I am removing the station IDs (for the station at the beginning of each trip) because I will be more interested in deriving the latitudes and longitudes of each station

In [ ]:
# Removing said columns
for dataset in pre_2018:

    dataset.drop(
        columns = ["trip_id", "bikeid", "tripduration", "from_station_id", "to_station_id", "birthyear", "gender"],
        inplace = True
    )

##### Renaming certain columns

In [ ]:
for dataset in pre_2018[:3]:

    # Rename some of the columns
    dataset.rename(
        columns = {
            "usertype" : "user_type",
            "starttime" : "start_time",
            "stoptime" : "stop_time"
        }, inplace = True
    )
    
trips_2017.rename(
    columns = {
        "end_time" : "stop_time",
        "usertype" : "user_type"
    }, inplace = True
)

##### Checking for Missing Values

In [ ]:
for dataset in pre_2018:

    print(dataset.isna().sum())
    print("######################")

##### Checking for Duplicates   

In [ ]:
# Use a list comprehension to view the number of duplicated observations in each dataset
[
    dataset[dataset.duplicated(keep = "last") == True].shape[0] for dataset in pre_2018
]

###### All four datasets from 2014 to 2017 contain duplicated observations, so we remove them in the next step

In [ ]:
for dataset in pre_2018:

    dataset.drop_duplicates(inplace = True)

In [ ]:
trips_pre_2018 = concat_to_parquet(list_of_dataframes = pre_2018, folder_name = CLEANED_DATA, parquet_name = "trips_pre_2018")

## 2018 - 2019

### 2018

In [ ]:
trips_2018 = pd.read_parquet(ALTERED_DATA_TYPES/"2018 - 2019/0.parquet")
trips_2018.columns

In [ ]:
trips_2018.drop(
    columns = [
        "Member Gender", "05 - Member Details Member Birthday Year", "trip_id", "from_station_id", "to_station_id",
        "01 - Rental Details Duration In Seconds Uncapped", "tripduration", "birthyear", "gender", "bikeid",
        "01 - Rental Details Rental ID", "03 - Rental Start Station ID", "02 - Rental End Station ID",
        "01 - Rental Details Bike ID"
    ], inplace = True
    )

#### Viewing the Data

In [ ]:
trips_2018.columns

###### Looking at the structure of 2018's dataframe, it would appear that this dataframe it is a diagonal matrix. It has 3,603,082 rows. Let us consider how many missing values there are in the "lower right" section of what appears to be a diagonal matrix above.

In [ ]:
trips_2018.head()

In [ ]:
trips_2018.iloc[:,6:].isna().sum()

###### As stated, I suspect that this is a diagonal matrix. To confirm this, I have to investigate the number of missing values before and after the line that divides the data into two vertical sections. We see for instance that there are 387,145 missing values in each of the rows of this "lower right" section of the matrix.

###### Let us check how many missing values there are on the "upper left" section of the matrix

In [ ]:
trips_2018.iloc[:,:6].isna().sum()

###### Notice that we would have a major piece of supporting evidence (in favour of my suspicion) if the sum of the number of missing values from the two sections equalled the total number of missing values in the whole matrix. This is exactly the case, since 3,215,937 + 387,145 = 3,603,082.

###### We need to find out on which row (of the "lower left" dataframe) the missing values start, so that we can see where this empty block of data begins. It would be a mistake to assume that the missing values on the "lower left" start at the half-way point (row-wise). 

In [ ]:
# Find the first row where a missing value occurs
trips_2018.apply(pd.Series.last_valid_index)

###### The first missing value occurs on row #387,145. And for further confirmation, let us check whether there are any non-missing values after this row.

In [ ]:
trips_2018.iloc[387145:,:].isna().sum()

###### Just to be thorough, let's see when the missing values on the "top left" end

In [ ]:
trips_2018.head()

In [ ]:
trips_2018.apply(pd.Series.first_valid_index)

In [ ]:
trips_2018.columns

In [ ]:
# Find the first row where a missing value occurs
right_half_2018 = [find_first_nan(data = trips_2018.iloc[:,i:], missing = False, just_reveal = False) for i in range(5,9)]
right_half_2018

###### They end at the same place where they begin on the "top right"

##### Forming the Final 2018 Dataset

In [ ]:
top_left_2018 = trips_2018.iloc[:387145, :5]
bottom_right_2018 = trips_2018.iloc[387145:, 5:]

# Final Renaming of columns
top_left_2018.rename(columns = {
                    "01 - Rental Details Local Start Time": "start_time",
                    "01 - Rental Details Local End Time": "end_time",
                    "03 - Rental Start Station Name": "from_station_name",
                    "02 - Rental End Station Name": "to_station_name",
                    "User Type": "user_type"}, inplace = True)


bottom_right_2018.rename(columns = {"usertype": "user_type"}, inplace = True)


In [ ]:
trips_2018 = pd.concat([top_left_2018, bottom_right_2018], axis = 0, ignore_index = True)

In [ ]:
trips_2018.columns

### 2019

#### Viewing the Data

In [ ]:
trips_2019 = pd.read_parquet(ALTERED_DATA_TYPES/"2018 - 2019/1.parquet")

In [ ]:
trips_2019.reset_index(inplace = True)
trips_2019.columns

###### Let us remove the columns that we are not going to use

In [ ]:
trips_2019.drop(
    columns = [
        "trip_id",  "gender", "birthyear", "01 - Rental Details Rental ID", "from_station_id",
        "01 - Rental Details Bike ID", "03 - Rental Start Station ID", "02 - Rental End Station ID",
        "Member Gender", "bikeid", "05 - Member Details Member Birthday Year", "to_station_id",
        "index", "01 - Rental Details Duration In Seconds Uncapped", "tripduration"
    ], inplace = True
)

In [ ]:
trips_2019.columns

##### Dealing With Missing Values

###### The 2019 dataframe has 3,818,004 rows. Let us check for the number of missing values 

In [ ]:
trips_2019.isna().sum()

###### As with the 2018 data, there are (in reality) only 5 columns here, but those columns have been duplicated under different names. The number of rows has been divided in the following manner: in the first 5 columns, 1,108,163 of the values are missing values. The second set of 5 columns (which I consider to be a duplicate of the first set) there are 2,709,841 missing values.

###### Let us have a look at the exact structure of these blocks of missing values is.

In [ ]:
trips_2019.apply(pd.Series.first_valid_index)

In [ ]:
# Produce a list consisting of the row index on which a NaN first occurs for each column
first_nan_locations_2019 = [
    find_first_nan(data = trips_2019.iloc[:,i:], missing = True, just_reveal = False) for i in range(0,10)
] 

In [ ]:
# View said list 
first_nan_locations_2019

###### The missing values in the first set of 5 columns begin on row #356,069. Those in the second set of 5 columns start from the beginning. So let us invert the question by asking when the non-missing values in the data start?

In [ ]:
# Produce a list consisting of the row index on which a non-NaN first occurs for each column
first_non_nan_locations_2019 = [
    find_first_nan(data = trips_2019.iloc[:,i:], missing = False, just_reveal = False) for i in range(0,10)
]

In [ ]:
# Again, view said list
first_non_nan_locations_2019

###### The non-missing values in the second set of 5 columns also begin on row #356,069. This suggests that the missing values in one set of 5 columns are possibly present in the other set. If true, this would mean that, as a result of the evident duplication of columns, there are in fact no missing values in the data.

###### Let us check whether the number of missing values remains constant for every row. This would lend some credence to our budding theory.

In [ ]:
for i in first_nan_locations_2019:

    row_nan_count = 0
    
    for j in range(0,10):

        if pd.isnull(trips_2019.iloc[i, j]):

            row_nan_count += 1

    print(f"There are {row_nan_count} missing values on row #{i}")

###### Every row that contains missing values contains exactly 5 of them. Let us go one step further to confirm the theory. I will begin to isolate the values in the left half of the dataset, and check whether they are all missing. We have already seen that there are 1,108,163 missing values in the left half of the data, and we have seen that the missing values start from row #365069 for its 5 columns. What we want to confirm now is whether or not these 1,108,163 missing values all occur exactly one after the other in a single unbroken sequence.

In [ ]:
# Isolate the missing values in the left half of the dataset
trips_2019.iloc[365069: 365069 + 1108163, :5].isna().sum()

###### Indeed they do. 

###### Let us investigate the missing values in the second half of the data. To begin with, we know that the first 365,069 values are missing. But we also know that there are 2,709,841 missing values in each column of the second half of the data. We know that 365,070 is not a missing value. But where are the remaining 2,344,772 values (per column)?

In [ ]:
trips_2019.iloc[365069:, :10].isna().sum()

In [ ]:
# Produce a list consisting of the row index on which a NaN first occurs for each column
find_first_nan(data = trips_2019.iloc[365069:, 5:10], missing = True, just_reveal = True)

In [ ]:
find_first_nan(data = trips_2019.iloc[1108163:, 5:10], missing = True, just_reveal = True)

###### We have found the remaining 2,344,772 missing values

In [ ]:
trips_2019.iloc[365069+1108163:, 5:10].isna().sum()

###### Below, we see that in the preceding five columns, these values are present in the same rows.

In [ ]:
trips_2019.iloc[365069+1108163:, :5].isna().sum()

##### Forming the Final 2019 Dataset

In [ ]:
trips_2019.columns

In [ ]:
trips_2019.rename(
    columns = {
        "usertype":"user_type",
        "end_time":"stop_time"
    }, inplace = True
)

In [ ]:
top_left_2019 = trips_2019.iloc[:365069,:5]
bottom_left_2019 = trips_2019.iloc[365069+1108163:,:5]
right_side_2019 = trips_2019.iloc[365069:365069+1108163,5:]

In [ ]:
right_side_2019.columns

###### We need to rename the columns 

In [ ]:
right_side_2019.rename(
    columns = {"01 - Rental Details Local Start Time": "start_time", 
               "01 - Rental Details Local End Time": "stop_time", 
               "03 - Rental Start Station Name": "from_station_name", 
               "02 - Rental End Station Name": "to_station_name",
               "User Type": "user_type" 
               }, inplace = True
)

In [ ]:
bottom_left_2019.columns

###### We attach these components together

In [ ]:
trips_2019

In [ ]:
trips_2019 = pd.concat(
    [
        top_left_2019, bottom_left_2019, right_side_2019
    ], axis = 0
)

###### I almost forgot to remove duplicate values

In [ ]:
trips_2019.drop_duplicates(inplace = True)

###### We need to make "trip_duration" a column of floats.

## Geocoding the data from 2014 - 2019

In [ ]:
pre_2020 = [trips_2014, trips_2015, trips_2016, trips_2017, trips_2018, trips_2019]

trips_pre_2020 = concat_to_parquet(list_of_dataframes = pre_2020, folder_name = CLEANED_DATA, parquet_name = "pre_2020")

In [ ]:
trips_pre_2020 = pd.read_parquet(CLEANED_DATA/"pre_2020.parquet")

### Points of Origin

###### It's important to have the unique station names for geocoding, lest we geocode a bunch of duplicates.

In [ ]:
origins = list(trips_pre_2020["from_station_name"].unique())

######  In fact, if we specify that we don't want duplicates, we get 743 station names, instead of 20,463,107 entries.
###### So we get the Nominatim geocoder to obtain the coordinates for each place, and associate each place with its precise location. Then we will need to put the latitudes and longitudes of each point in their respective columns of the dataframe.


In [ ]:
places_and_points = use_primary_geocoder(places = origins)

In [ ]:
add_coordinates_to_dataframe(
    data = trips_pre_2020,
    places_and_points = places_and_points,
    start_or_stop = "start"
)

### Destinations

In [ ]:
destinations = list(
    trips_pre_2020["to_station_name"].unique()
)

###### If there are some destinations which are not among the points of origin, then we will have to geocode them. Otherwise, there'll be no need for further geocoding. So let us check for the presence of any such destinations.

In [ ]:
not_in_common = [origin for origin in origins if origin not in destinations]
len(not_in_common)

###### There are no such destinations, and consequently, the "to_station_name" column does not need to be geocoded.

In [ ]:
add_coordinates_to_dataframe(
    data = trips_pre_2020,
    places_and_points = places_and_points,
    start_or_stop = "stop"
)

In [ ]:
trips_pre_2020.head()

In [ ]:
trips_pre_2020.to_parquet(path = TEMPORARY_DATA/"geocoded_pre_2020.parquet")

## Final Clean of the data from 2014-2019

In [ ]:
trips_pre_2020 = pd.read_parquet(TEMPORARY_DATA/"geocoded_pre_2020.parquet")

### Dealing with the rows which contain station names that could not be geocoded

#### Finding all the rows (by indices) which contain data that could not be geocoded.

In [ ]:
trips_pre_2020.head()

##### Origins

In [ ]:
rows_with_unknown_start_lats = find_rows_with_zeros(data = trips_pre_2020, column_index = 5)
rows_with_unknown_start_lngs = find_rows_with_zeros(data = trips_pre_2020, column_index = 6)

###### Find out how many such rows there are. 

In [ ]:
print(len(rows_with_unknown_start_lats), len(rows_with_unknown_start_lngs))

###### We have the same number of rows for which the latitudes and longitudes of the points of origin were not obtained. This makes sense, since it is probably unlikely for the geocoder to work for one coordinate, but not the other. To confirm this, let us check whether the rows for which the latitudes and longitudes were not geocoded are exactly the same.

In [ ]:
len(
    [row for row in rows_with_unknown_start_lats if row in rows_with_unknown_start_lngs]
)

##### Destinations

In [ ]:
rows_with_unknown_stop_lats = find_rows_with_zeros(data = trips_pre_2020, column_index = 7)
rows_with_unknown_stop_lngs = find_rows_with_zeros(data = trips_pre_2020, column_index = 8)

In [ ]:
print(len(rows_with_unknown_stop_lats), len(rows_with_unknown_stop_lngs))

###### Again, we have the same number of rows for which the latitudes and longitudes of the station names were not obtained. To confirm this, let us check whether the rows for which the latitudes and longitudes were not geocoded are exactly the same.

In [ ]:
len(
    [row for row in rows_with_unknown_start_lats if row in rows_with_unknown_start_lngs]
)

#### Using a secondary geocoder (Photon) to geocode the outstanding points of origin.

###### The alternative would have been to delete all rows in which the origin and destination stations were unknown

In [ ]:
non_geocoded_origins = list(
    trips_pre_2020.iloc[rows_with_unknown_start_lats, 2].unique()
)

##### Geocoding

In [ ]:
remaining_origins_with_points = use_secondary_geocoder(
    data = trips_pre_2020,
    column_of_station_names = "from_station_name",
    row_indices = rows_with_unknown_start_lats
)

###### Placing the obtained coordinates of the origins in the appropriate columns

In [ ]:
for row, point in zip(
        rows_with_unknown_start_lats, remaining_origins_with_points.values()
):
    
    trips_pre_2020["start_latitude"].replace(trips_pre_2020.iloc[row, 5], point[0])
    trips_pre_2020["start_longitude"].replace(trips_pre_2020.iloc[row, 6], point[1])

###### Saving the current state of the data to save time

#### Using the Photon geocoder to geocode the outstanding destinations.

###### Geocoding

In [ ]:
remaining_destinations_with_points = use_secondary_geocoder(
    data = trips_pre_2020,
    column_of_station_names = "to_station_name",
    row_indices = rows_with_unknown_stop_lats
)

###### Placing the obtained coordinates of the destinations in the appropriate columns

In [ ]:
for row, point in zip(
        rows_with_unknown_stop_lats, remaining_destinations_with_points.values()
):

    trips_pre_2020["stop_latitude"].replace(trips_pre_2020.iloc[row, 7], point[0])
    trips_pre_2020["stop_longitude"].replace(trips_pre_2020.iloc[row, 8], point[1])

###### Save the current state of the data

In [ ]:
trips_pre_2020.to_parquet(path = CLEANED_DATA/"pre_2020.parquet")

#### Replace the last few latitudes and longitudes that were not geocoded

In [ ]:
trips_pre_2020 = pd.read_parquet(CLEANED_DATA/"pre_2020.parquet")

In [ ]:
final_unknown_origin_lats = reveal_final_unknown_lats(data = trips_pre_2020, column_of_coordinate = "start_latitude")

In [ ]:
len(final_unknown_origin_lats)

##### Many of the latitudes are 41, and many of the longitudes are around -87

In [ ]:
for column in ["start_latitude", "start_longitude"]:

    trips_pre_2020[column].replace(0.0, 41, inplace = True)


for column in ["stop_latitude", "stop_longitude"]:

    trips_pre_2020[column].replace(0.0, -87, inplace = True)

In [ ]:
trips_pre_2020.drop(columns = ["from_station_name", "to_station_name", "user_type"], inplace = True)
trips_pre_2020.drop_duplicates(inplace = True)

In [ ]:
trips_pre_2020 = trips_pre_2020.to_parquet(path = CLEANED_DATA/"final_pre_2020.parquet")

## 2020 - 2023

In [4]:
trips_pre_2020 = pd.read_parquet(path = CLEANED_DATA/"final_pre_2020.parquet")

In [6]:
trips_2020 = pd.read_parquet(ALTERED_DATA_TYPES/"2020 - 2023/0.parquet")
trips_2021 = pd.read_parquet(ALTERED_DATA_TYPES/"2020 - 2023/1.parquet")
trips_2022 = pd.read_parquet(ALTERED_DATA_TYPES/"2020 - 2023/2.parquet")
trips_2023 = pd.read_parquet(ALTERED_DATA_TYPES/"2020 - 2023/3.parquet")

### Removing some unnecessary features, and renaming others

In [7]:
from_2020 = [trips_2020, trips_2021, trips_2022, trips_2023]

for dataset in from_2020:

    #  I no longer need the station names
    dataset.drop(columns = ["ride_id", "rideable_type", "member_casual"], inplace = True)

    dataset.rename(
        columns = {
            "started_at": "start_time", 
            "ended_at" : "stop_time",
            "start_station_name" : "from_station_name",
            "end_station_name" : "to_station_name", 
            "start_lat" : "start_latitude",
            "start_lng" : "start_longitude",
            "end_lat" : "stop_latitude",
            "end_lng" : "stop_longitude",
            },
        inplace = True
    )

#### Checking out the missing values

###### For each year, there are a couple of thousand trips for which the destination's coordinates and names are unknown.

In [8]:
trips_from_2020 = pd.concat(from_2020)

In [9]:
trips_from_2020.isna().sum()

start_time                 0
stop_time                  0
from_station_name    2458535
to_station_name      2634071
start_latitude             0
start_longitude            0
stop_latitude          21635
stop_longitude         21635
dtype: int64

###### Let us see whether there are a siginificant number of instances where the destination names are present, without correponding coordinates. If so, geocoding will be justified.

In [10]:
trips_from_2020[(pd.isnull(trips_from_2020["to_station_name"])) & (pd.isnull(trips_from_2020["stop_latitude"])) & 
(pd.isnull(trips_from_2020["stop_longitude"])) ]

,start_time,stop_time,from_station_name,to_station_name,start_latitude,start_longitude,stop_latitude,stop_longitude
414426,2020-03-16 11:23:36,2020-03-16 11:23:24,HQ QR,NaN,41.889900,-87.680300,NaN,NaN
1001,2020-04-07 11:53:08,2020-04-07 12:28:35,Wells St & Concord Ln,NaN,41.912100,-87.634700,NaN,NaN
1864,2020-04-20 12:24:48,2020-04-20 12:29:46,Racine Ave & Wrightwood Ave,NaN,41.928900,-87.659000,NaN,NaN
2167,2020-04-16 08:41:56,2020-04-16 11:33:48,Racine Ave & 18th St,NaN,41.858200,-87.656500,NaN,NaN
2458,2020-04-09 15:33:45,2020-04-09 16:34:54,Morgan Ave & 14th Pl,NaN,41.862400,-87.651100,NaN,NaN
...,...,...,...,...,...,...,...,...
356624,2023-11-23 19:51:43,2023-11-24 20:51:39,Clark St & Armitage Ave,NaN,41.918306,-87.636282,NaN,NaN
356625,2023-11-23 19:49:48,2023-11-24 20:49:42,Clark St & Armitage Ave,NaN,41.918306,-87.636282,NaN,NaN
356661,2023-11-09 15:37:23,2023-11-10 16:37:19,Clark St & Jarvis Ave,NaN,42.015963,-87.675005,NaN,NaN
356677,2023-11-16 08:05:05,2023-11-17 09:04:58,Clark St & Wellington Ave,NaN,41.936497,-87.647539,NaN,NaN


###### In almost all instances, both the destination names and the corresponding coordinates are missing. Therefore, it is justified that I replace the missing coordinates with likely values (as I did before)

In [11]:
for column in ["start_latitude", "start_longitude"]:

    trips_from_2020[column].fillna(value = 41, inplace = True)

for column in ["stop_latitude", "stop_longitude"]:

    trips_from_2020[column].fillna(value = -87, inplace = True)

###### I don't need the names of the origins and destinations

In [12]:
trips_from_2020.drop(
        columns = ["from_station_name", "to_station_name"],
        inplace = True
    )

In [13]:
trips_from_2020

,start_time,stop_time,start_latitude,start_longitude,stop_latitude,stop_longitude
0,2020-01-21 20:06:59,2020-01-21 20:14:30,41.966500,-87.688400,41.967100,-87.667400
1,2020-01-30 14:22:39,2020-01-30 14:26:22,41.961600,-87.666000,41.954200,-87.664400
2,2020-01-09 19:29:26,2020-01-09 19:32:17,41.940100,-87.645500,41.940200,-87.653000
3,2020-01-06 16:17:07,2020-01-06 16:25:56,41.884600,-87.631900,41.891800,-87.620600
4,2020-01-30 08:37:16,2020-01-30 08:42:48,41.885600,-87.641800,41.889900,-87.634300
...,...,...,...,...,...,...
362513,2023-11-24 08:39:27,2023-11-24 08:47:03,41.936497,-87.647539,41.935775,-87.663600
362514,2023-11-06 09:07:20,2023-11-06 09:10:00,41.877726,-87.654787,41.877642,-87.649618
362515,2023-11-10 19:35:30,2023-11-10 19:44:28,41.943687,-87.648855,41.935775,-87.663600
362516,2023-11-27 09:11:23,2023-11-27 09:13:23,41.877726,-87.654787,41.877642,-87.649618


##### Duplicate Values

In [14]:
trips_from_2020.drop_duplicates(inplace = True)

## Final Data

In [15]:
all_data = pd.concat([trips_pre_2020, trips_from_2020], axis = 0)

In [16]:
all_data.to_parquet(path = CLEANED_DATA/"final.parquet")

In [19]:
all_data

,start_time,stop_time,start_latitude,start_longitude,stop_latitude,stop_longitude
0,2014-06-30 23:57:00,2014-07-01 00:07:00,41.939304,-87.668278,41.945514,-87.646477
1,2014-06-30 23:56:00,2014-07-01 00:00:00,41.864819,-87.647128,41.869388,-87.655475
2,2014-06-30 23:33:00,2014-06-30 23:35:00,41.921687,-87.653714,41.919936,-87.648830
3,2014-06-30 23:26:00,2014-07-01 00:24:00,41.877702,-87.649654,49.318630,11.131904
4,2014-06-30 23:16:00,2014-06-30 23:26:00,41.872165,-87.661434,41.877702,-87.649654
...,...,...,...,...,...,...
362513,2023-11-24 08:39:27,2023-11-24 08:47:03,41.936497,-87.647539,41.935775,-87.663600
362514,2023-11-06 09:07:20,2023-11-06 09:10:00,41.877726,-87.654787,41.877642,-87.649618
362515,2023-11-10 19:35:30,2023-11-10 19:44:28,41.943687,-87.648855,41.935775,-87.663600
362516,2023-11-27 09:11:23,2023-11-27 09:13:23,41.877726,-87.654787,41.877642,-87.649618
